In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB

In [2]:
newsgroup_train = fetch_20newsgroups(subset='train')
newsgroup_test = fetch_20newsgroups(subset='test')
X_train = newsgroup_train.data
y_train = newsgroup_train.target
X_test = newsgroup_test.data
y_test = newsgroup_test.target

In [11]:
dict_classifiers = {
    "Nearest Neighbors": KNeighborsClassifier(n_neighbors=5, weights='distance')
#    "Linear SVM": SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=5, random_state=42),
#    "Decision Tree": tree.DecisionTreeClassifier(),
 #   "Neural Net": MLPClassifier(alpha = 1),
 #   "Naive Bayes": MultinomialNB()
}

In [12]:
for name, classifier in dict_classifiers.items():
    text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer()), ('clf', classifier)])
    text_clf = text_clf.fit(X_train, y_train)
    predicted = text_clf.predict(X_test)
    print("Accuracy of "+ name + " : "+str(np.mean(predicted == y_test)))

Accuracy of Nearest Neighbors : 0.697689856612
